In [1]:
import pandas as pd
import cufflinks as cf

# Настройка режима офлайн для работы с cufflinks и plotly
cf.go_offline()

# Загрузка данных из файла
data = pd.read_csv('Sunspots.csv', low_memory=False, infer_datetime_format=True, index_col=['Date'], dayfirst=True)
data = data.rename(columns={'Monthly Mean Total Sunspot Number':'Sunspots'})
data = data.drop('Unnamed: 0', axis=1)
# Визуализация данных (например, потребление энергии по глобальной активной мощности)
data['Sunspots'].iplot(title='Sunspots', xTitle='Date', yTitle='Sunspots')


C:\Users\romar\AppData\Local\Temp\ipykernel_13080\65387575.py:8: FutureWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



In [2]:
data

,Sunspots
Date,
1749-01-31,96.7
1749-02-28,104.3
1749-03-31,116.7
1749-04-30,92.8
1749-05-31,141.7
...,...
2020-09-30,0.6
2020-10-31,14.4
2020-11-30,34.0


In [3]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [4]:
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_data = scaler.fit_transform(data[['Sunspots']])
scaled_data = data[['Sunspots']].values

In [5]:
scaled_data

array([[ 96.7],
       [104.3],
       [116.7],
       ...,
       [ 34. ],
       [ 21.8],
       [ 10.4]])

In [6]:
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 114  # Например, используем последние 10 значений для прогнозирования следующего
x, y = create_dataset(scaled_data, look_back)




In [7]:
X_dif = x[:,:-1] - x[:,1:]

In [8]:
X_dif.shape

(3151, 113)

In [9]:
pd.DataFrame(x)

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,96.7,104.3,116.7,92.8,141.7,139.2,158.0,110.5,126.5,125.8,...,66.2,54.2,107.8,55.8,62.7,86.7,81.7,120.5,77.3,75.0
1,104.3,116.7,92.8,141.7,139.2,158.0,110.5,126.5,125.8,264.3,...,54.2,107.8,55.8,62.7,86.7,81.7,120.5,77.3,75.0,73.3
2,116.7,92.8,141.7,139.2,158.0,110.5,126.5,125.8,264.3,142.0,...,107.8,55.8,62.7,86.7,81.7,120.5,77.3,75.0,73.3,64.5
3,92.8,141.7,139.2,158.0,110.5,126.5,125.8,264.3,142.0,122.2,...,55.8,62.7,86.7,81.7,120.5,77.3,75.0,73.3,64.5,104.2
4,141.7,139.2,158.0,110.5,126.5,125.8,264.3,142.0,122.2,126.5,...,62.7,86.7,81.7,120.5,77.3,75.0,73.3,64.5,104.2,62.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3146,78.6,76.1,58.2,56.1,64.5,65.8,120.1,125.7,139.1,109.3,...,0.5,1.5,6.2,0.2,1.5,5.2,0.2,5.8,6.1,7.5
3147,76.1,58.2,56.1,64.5,65.8,120.1,125.7,139.1,109.3,94.4,...,1.5,6.2,0.2,1.5,5.2,0.2,5.8,6.1,7.5,0.6
3148,58.2,56.1,64.5,65.8,120.1,125.7,139.1,109.3,94.4,47.8,...,6.2,0.2,1.5,5.2,0.2,5.8,6.1,7.5,0.6,14.4
3149,56.1,64.5,65.8,120.1,125.7,139.1,109.3,94.4,47.8,86.6,...,0.2,1.5,5.2,0.2,5.8,6.1,7.5,0.6,14.4,34.0


In [10]:
x.shape, X_dif.shape

((3151, 114), (3151, 113))

In [11]:
X_new = np.hstack([x,X_dif])
X_new.shape

(3151, 227)

In [12]:
y.shape

(3151,)

In [13]:
# Преобразование в форму, которую ожидает RNN
# x = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))
x = X_new.copy()

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)


In [15]:
scaler_x = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))
x_train = scaler_x.fit_transform(x_train)
x_test = scaler_x.transform(x_test)
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test = scaler_y.transform(y_test.reshape(-1, 1))

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))



In [16]:
min(scaler_x.data_min_), max(scaler_x.data_max_)

(-156.5, 398.2)

In [17]:
min(scaler_y.data_min_), max(scaler_y.data_max_)

(0.0, 398.2)

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense


In [19]:
model = Sequential()

# Входной слой
model.add(SimpleRNN(50, activation='relu', input_shape=(x_train.shape[1], 1), return_sequences=True))

# Дополнительные слои (если необходимо)
model.add(SimpleRNN(50, activation='relu', return_sequences=True))
model.add(SimpleRNN(50, activation='relu'))

# Выходной слой
model.add(Dense(1))


In [20]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [21]:
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))
# на relu ошибка уменьшается медленнее


Epoch 1/20
79/79 [==============================] - 18s 176ms/step - loss: 0.0231 - val_loss: 0.0133
Epoch 2/20
79/79 [==============================] - 13s 162ms/step - loss: 0.0149 - val_loss: 0.0131
Epoch 3/20
79/79 [==============================] - 12s 151ms/step - loss: 0.0129 - val_loss: 0.0098
Epoch 4/20
79/79 [==============================] - 11s 135ms/step - loss: 0.0127 - val_loss: 0.0159
Epoch 5/20
79/79 [==============================] - 11s 146ms/step - loss: 0.0117 - val_loss: 0.0149
Epoch 6/20
79/79 [==============================] - 12s 148ms/step - loss: 0.0109 - val_loss: 0.0090
Epoch 7/20
79/79 [==============================] - 13s 165ms/step - loss: 0.0104 - val_loss: 0.0098
Epoch 8/20
79/79 [==============================] - 12s 156ms/step - loss: 0.0099 - val_loss: 0.0108
Epoch 9/20
79/79 [==============================] - 11s 139ms/step - loss: 0.0085 - val_loss: 0.0124
Epoch 10/20
79/79 [==============================] - 9s 119ms/step - loss: 0.0082 - val_los

In [22]:
predictions = model.predict(x_test)
predicted_power = scaler_y.inverse_transform(predictions)  # Если вы ранее использовали нормализацию, то выполните обратное преобразование.

# Затем вы можете сравнить "predicted_power" с вашими исходными данными и оценить точность прогноза.


20/20 [==============================] - 2s 36ms/step


In [23]:
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler_y.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

In [24]:
from sklearn.metrics import mean_absolute_error
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred+0.001) / (y_true+0.001))) * 100
mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)
print(f"MAPE: {mape}%")
print(f"MAE: {mae}")

MAPE: 2589.4325103171473%
MAE: 33.879211069097984


SimpleRNN в Keras является одним из слоев для рекуррентной обработки, и у него есть множество атрибутов и параметров, которые можно настроить. Ниже приведены основные атрибуты и их описания:

units: Положительное целое число, определяющее размерность выходного пространства (т.е. количество нейронов в слое).

activation: Функция активации для использования в ячейке. По умолчанию 'tanh'.

use_bias: Логическое значение, указывающее, следует ли слою использовать смещение.

kernel_initializer: Инициализатор для весов матрицы ядра.

recurrent_initializer: Инициализатор для весов рекуррентной матрицы.

bias_initializer: Инициализатор смещения.

kernel_regularizer: Функция регуляризации, применяемая к матрице ядра.

recurrent_regularizer: Функция регуляризации, применяемая к рекуррентной матрице.

bias_regularizer: Функция регуляризации, применяемая к вектору смещения.

activity_regularizer: Функция регуляризации, применяемая к выходу слоя.

kernel_constraint: Ограничение, применяемое к матрице ядра.

recurrent_constraint: Ограничение, применяемое к рекуррентной матрице.

bias_constraint: Ограничение, применяемое к вектору смещения.

dropout: Доля входных единиц для отсева на каждом обновлении во время обучения.

recurrent_dropout: Доля рекуррентных единиц для отсева на каждом обновлении во время обучения.

return_sequences: Логическое значение. Если истина, возвращает полную последовательность к последнему слою для каждого входа. Если ложь, возвращает только последний выход.

return_state: Логическое значение. Если истина, кроме выходных значений, возвращает также последние состояния.

go_backwards: Логическое значение. Если истина, обрабатывает входные последовательности в обратном порядке.

stateful: Логическое значение. Если истина, слой будет рассматриваться как состояние, которое может быть передано в следующий слой.

unroll: Логическое значение. Если истина, сеть будет раскрывать рекуррентные циклы, что может ускорить процесс, но при этом будет использовать больше памяти.

input_dim: Размерность входных данных. Обычно не требуется, если первым слоем модели является слой Embedding или другой слой, который определяет input_shape.

input_length: Длина входных последовательностей. Обычно не требуется, если первым слоем модели является слой Embedding.

In [25]:
from tensorflow.keras.layers import LSTM, Dense

In [26]:
model = Sequential()

# Входной слой LSTM
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))

# Дополнительные слои (если необходимо)
model.add(LSTM(50, activation='tanh', return_sequences=True))
model.add(LSTM(50, activation='tanh'))

# Выходной слой
model.add(Dense(1))


In [27]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [28]:
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/20
79/79 [==============================] - 28s 257ms/step - loss: 0.0309 - val_loss: 0.0313
Epoch 2/20
79/79 [==============================] - 18s 222ms/step - loss: 0.0732 - val_loss: 0.0309
Epoch 3/20
79/79 [==============================] - 19s 236ms/step - loss: 0.0298 - val_loss: 0.0307
Epoch 4/20
79/79 [==============================] - 18s 233ms/step - loss: 0.0294 - val_loss: 0.0311
Epoch 5/20
79/79 [==============================] - 19s 242ms/step - loss: 0.0300 - val_loss: 0.0304
Epoch 6/20
79/79 [==============================] - 21s 267ms/step - loss: 0.0284 - val_loss: 0.0279
Epoch 7/20
79/79 [==============================] - 19s 237ms/step - loss: 0.0197 - val_loss: 0.0111
Epoch 8/20
79/79 [==============================] - 18s 229ms/step - loss: 0.0142 - val_loss: 0.0111
Epoch 9/20
79/79 [==============================] - 18s 228ms/step - loss: 0.0120 - val_loss: 0.0100
Epoch 10/20
79/79 [==============================] - 18s 230ms/step - loss: 0.0097 - val_lo

In [29]:
predictions = model.predict(x_test)
predicted_power = scaler_y.inverse_transform(predictions)  # Если вы ранее использовали нормализацию, то выполните обратное преобразование.

# Затем вы можете сравнить "predicted_power" с вашими исходными данными и оценить точность прогноза.


20/20 [==============================] - 2s 61ms/step


In [30]:
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler_y.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

В TensorFlow (с использованием Keras API), слой LSTM имеет множество атрибутов и параметров. Ниже приведены некоторые из них:

units: Количество LSTM ячеек (или скрытых состояний) в слое.

activation: Функция активации, используемая для обновления скрытых состояний. По умолчанию это "tanh".

recurrent_activation: Функция активации, используемая для забывания, входного и выходного вентилей. По умолчанию это "sigmoid".

use_bias: Булево значение, указывающее, следует ли использовать смещение в слое.

kernel_initializer, recurrent_initializer, bias_initializer: Инициализаторы, которые определяют стратегию инициализации весов.

dropout, recurrent_dropout: Процент отсева для входных данных и рекуррентных данных соответственно.

return_sequences: Булево значение. Если True, возвращает весь последовательный выход; если False, возвращает только последний выход.

return_state: Булево значение. Если True, возвращает состояние вдобавок к выходу.

go_backwards: Булево значение. Если True, обрабатывает входные последовательности в обратном порядке.

stateful: Булево значение. Если True, слой будет использовать состояния во время обучения и прогнозирования.

unroll: Булево значение. Если True, сеть будет развертывать рекурсию, что может ускорить процесс обучения, но потребует больше памяти.

implementation: Один из {1, 2}. Рекомендуется использовать 2, так как это более эффективная реализация, особенно для GPU.

In [31]:
# from sklearn.metrics import mean_absolute_error
# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred+0.001) / (y_true+0.001))) * 100
mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)

print(f"MAPE: {mape}%")
print(f"MAE: {mae}")

MAPE: 1825.587675024019%
MAE: 19.86750004537132


In [32]:
from tensorflow.keras.layers import GRU, Dense

In [33]:
model = Sequential()

# Входной слой GRU
model.add(GRU(50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))

# Дополнительные слои (если необходимо)
model.add(GRU(50, return_sequences=True))
model.add(GRU(50))

# Выходной слой
model.add(Dense(1))


In [34]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [35]:
model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/20
79/79 [==============================] - 25s 237ms/step - loss: 0.0301 - val_loss: 0.0286
Epoch 2/20
79/79 [==============================] - 18s 229ms/step - loss: 0.0216 - val_loss: 0.0223
Epoch 3/20
79/79 [==============================] - 19s 236ms/step - loss: 0.0142 - val_loss: 0.0090
Epoch 4/20
79/79 [==============================] - 18s 231ms/step - loss: 0.0123 - val_loss: 0.0105
Epoch 5/20
79/79 [==============================] - 18s 232ms/step - loss: 0.0132 - val_loss: 0.0075
Epoch 6/20
79/79 [==============================] - 20s 256ms/step - loss: 0.0120 - val_loss: 0.0101
Epoch 7/20
79/79 [==============================] - 19s 237ms/step - loss: 0.0115 - val_loss: 0.0075
Epoch 8/20
79/79 [==============================] - 18s 232ms/step - loss: 0.0107 - val_loss: 0.0101
Epoch 9/20
79/79 [==============================] - 18s 234ms/step - loss: 0.0109 - val_loss: 0.0087
Epoch 10/20
79/79 [==============================] - 20s 250ms/step - loss: 0.0103 - val_lo

In [36]:
predictions = model.predict(x_test)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler_y.inverse_transform(predictions)


20/20 [==============================] - 2s 48ms/step


In [37]:
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler_y.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

In [38]:
# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)

print(f"MAPE: {mape}%")
print(f"MAE: {mae}")


MAPE: 2470.9150637333446%
MAE: 25.20235107198192


In [39]:
np.min(y_test_inv )

0.0

units: Количество GRU ячеек (или скрытых состояний) в слое.

activation: Функция активации для обновления скрытых состояний. По умолчанию "tanh".

recurrent_activation: Функция активации, используемая для обновления и сброса вентилей. По умолчанию "sigmoid".

use_bias: Булево значение, определяющее, следует ли использовать смещение в слое.

kernel_initializer, recurrent_initializer, bias_initializer: Инициализаторы для инициализации различных весовых матриц и смещений.

dropout, recurrent_dropout: Процент отсева для входных данных и рекуррентных данных соответственно.

return_sequences: Булево значение. Если True, возвращает весь последовательный выход; если False, возвращает только последний выход.

return_state: Булево значение. Если True, возвращает состояние вдобавок к выходу.

go_backwards: Булево значение. Если True, обрабатывает входные последовательности в обратном порядке.

stateful: Булево значение. Если True, слой будет сохранять свое состояние между пакетами, что позволяет обрабатывать длинные последовательности.

unroll: Булево значение. Если True, сеть будет развертывать рекурсию, что может ускорить процесс обучения, но потребует больше памяти.

implementation: Один из {1, 2}. Рекомендуется использовать 2, так как это более эффективная реализация, особенно для GPU.

reset_after: Булево значение. Определяет, когда следует применять сброс вентиля (перед или после матричного умножения). Это связано с различием между оригинальной версией GRU и версией, предложенной в "Learning to Execute".

In [40]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AttentionMechanism(Layer):
    def __init__(self, return_sequences=True):
        super(AttentionMechanism, self).__init__()
        self.return_sequences = return_sequences
        
    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
    def call(self, x):
        e = tf.nn.tanh(tf.matmul(x,self.W)+self.b)
        a = tf.nn.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return tf.reduce_sum(output, axis=1)


In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(AttentionMechanism(return_sequences=True))  # Внимание после LSTM
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')


In [42]:
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/30
79/79 [==============================] - 19s 174ms/step - loss: 0.0334 - val_loss: 0.0307
Epoch 2/30
79/79 [==============================] - 12s 156ms/step - loss: 0.0294 - val_loss: 0.0312
Epoch 3/30
79/79 [==============================] - 13s 162ms/step - loss: 0.0295 - val_loss: 0.0307
Epoch 4/30
79/79 [==============================] - 13s 164ms/step - loss: 0.0294 - val_loss: 0.0307
Epoch 5/30
79/79 [==============================] - 13s 159ms/step - loss: 0.0293 - val_loss: 0.0309
Epoch 6/30
79/79 [==============================] - 12s 154ms/step - loss: 0.0294 - val_loss: 0.0307
Epoch 7/30
79/79 [==============================] - 12s 156ms/step - loss: 0.0294 - val_loss: 0.0307
Epoch 8/30
79/79 [==============================] - 14s 179ms/step - loss: 0.0294 - val_loss: 0.0312
Epoch 9/30
79/79 [==============================] - 14s 181ms/step - loss: 0.0297 - val_loss: 0.0308
Epoch 10/30
79/79 [==============================] - 14s 183ms/step - loss: 0.0293 - val_lo

In [43]:
predictions = model.predict(x_test)


20/20 [==============================] - 2s 43ms/step


In [44]:
predictions = model.predict(x_test)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler_y.inverse_transform(predictions)

20/20 [==============================] - 1s 44ms/step


In [45]:
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler_y.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

In [46]:
# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)

print(f"MAPE: {mape}%")
print(f"MAE: {mae}")

MAPE: 13788.916573393464%
MAE: 58.85446911069746
